# 数据导入与预处理

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

font_path = '/System/Library/Fonts/Hiragino Sans GB.ttc' # 解决中文乱码问题，系统的中文字体，这是mac实现，如果是Window需要切换目录

sns.set_theme(font="Hiragino Sans GB")

# 导入数据
data = pd.read_excel('data-seg.xlsx')

## 需求
1. 项目支撑数：有商机编码无合同金额；
2. 支撑项目数：以商机编码为准，不重复计算，若解决方案经理与项目经理为同一人的算一个；
3. 支撑项目金额：不重复计算，以商机编码为准；
4. 看考虑那种形式方便后续统计，（如此次整理后，后续我们只需要导入几个字段的数据，就能统计出每个人的支撑项目数、签约项目数、签约金额）；


In [4]:
data.head(2)

,商机编码,合同总金额（元）,项目经理,项目经理协办人,解决方案经理,集成项目经理,交付经理,支撑人员
0,a1,0,a,g,h,h,NaN,NaN
1,a2,222,b,NaN,h,h,NaN,NaN


In [5]:
# 判断是否重复，结果是没有
data[data['商机编码'].duplicated(keep=False)]

,商机编码,合同总金额（元）,项目经理,项目经理协办人,解决方案经理,集成项目经理,交付经理,支撑人员


In [6]:
# 使用 melt 将人员列转换为行
melted_df = data.melt(id_vars=['商机编码', '合同总金额（元）'],value_vars=['项目经理', '项目经理协办人', '解决方案经理', '集成项目经理', '交付经理', '支撑人员'], var_name='人员岗位', value_name='人员名称')
melted_df.head()

,商机编码,合同总金额（元）,人员岗位,人员名称
0,a1,0,项目经理,a
1,a2,222,项目经理,b
2,a3,333,项目经理,c
3,a4,444,项目经理,d
4,a5,555,项目经理,e


In [7]:
# melted_df['人员名称'].isna()

# 去除指定列（例如 '商机编码' 和 '合同总金额（元）'）中有空值的行
df_cleaned = melted_df.dropna(subset=['人员名称'])
df_cleaned

,商机编码,合同总金额（元）,人员岗位,人员名称
0,a1,0,项目经理,a
1,a2,222,项目经理,b
2,a3,333,项目经理,c
3,a4,444,项目经理,d
4,a5,555,项目经理,e
5,a6,666,项目经理,f
6,a1,0,项目经理协办人,g
12,a1,0,解决方案经理,h
13,a2,222,解决方案经理,h
14,a3,333,解决方案经理,h


In [8]:
# 有些人员名称有多个
df_cleaned[df_cleaned['人员名称'].str.contains(',')]

,商机编码,合同总金额（元）,人员岗位,人员名称


In [9]:
# 拆分人员名称列
df_cleaned['人员名称'] = df_cleaned['人员名称'].str.split(',')

df_expanded = df_cleaned.explode('人员名称')
df_expanded

,商机编码,合同总金额（元）,人员岗位,人员名称
0,a1,0,项目经理,a
1,a2,222,项目经理,b
2,a3,333,项目经理,c
3,a4,444,项目经理,d
4,a5,555,项目经理,e
5,a6,666,项目经理,f
6,a1,0,项目经理协办人,g
12,a1,0,解决方案经理,h
13,a2,222,解决方案经理,h
14,a3,333,解决方案经理,h


In [10]:
# 确认没有未分割的人
df_expanded[df_expanded['人员名称'].str.contains(',')]

,商机编码,合同总金额（元）,人员岗位,人员名称


In [11]:
# 去掉岗位
df_expanded_clean = df_expanded.drop(columns=['人员岗位'])
df_expanded_clean

,商机编码,合同总金额（元）,人员名称
0,a1,0,a
1,a2,222,b
2,a3,333,c
3,a4,444,d
4,a5,555,e
5,a6,666,f
6,a1,0,g
12,a1,0,h
13,a2,222,h
14,a3,333,h


In [12]:
# 去掉重复数据
df_expanded_clean_dropdup = df_expanded_clean.drop_duplicates()
df_expanded_clean_dropdup

,商机编码,合同总金额（元）,人员名称
0,a1,0,a
1,a2,222,b
2,a3,333,c
3,a4,444,d
4,a5,555,e
5,a6,666,f
6,a1,0,g
12,a1,0,h
13,a2,222,h
14,a3,333,h


In [13]:
# 支撑项目数(bpm)
df_zero = df_expanded_clean_dropdup[df_expanded_clean_dropdup['合同总金额（元）'] == 0].groupby(['人员名称'])['商机编码'].count()
df_zero

人员名称
a    1
g    1
h    1
Name: 商机编码, dtype: int64

In [14]:
# 签约合同数量
df_contact = df_expanded_clean_dropdup[df_expanded_clean_dropdup['合同总金额（元）'] != 0].groupby(['人员名称'])['商机编码'].count()
df_contact

人员名称
b    1
c    1
d    1
e    1
f    1
h    4
Name: 商机编码, dtype: int64

In [15]:
# 签约合同金额
df_money = df_expanded_clean_dropdup.groupby(['人员名称'])['合同总金额（元）'].sum()
df_money

人员名称
a       0
b     222
c     333
d     444
e     555
f     666
g       0
h    1554
Name: 合同总金额（元）, dtype: int64

In [16]:
df_all = pd.concat([df_zero, df_contact, df_money], axis=1)
df_all = df_all.fillna(0)
df_all.columns = ['支撑项目数(bpm)', '签约合同数量', '合同总金额（元）']
df_all['支撑项目数(bpm)'] = df_all['支撑项目数(bpm)'].astype(int)
df_all['签约合同数量'] = df_all['签约合同数量'].astype(int)

df_all


,支撑项目数(bpm),签约合同数量,合同总金额（元）
人员名称,,,
a,1,0,0
g,1,0,0
h,1,4,1554
b,0,1,222
c,0,1,333
d,0,1,444
e,0,1,555
f,0,1,666


In [17]:
# 将DataFrame保存为Excel文件
df_all.to_excel('result.xlsx', index=True)